In [3]:
import os
import pickle
import torch
import numpy as np
import glob
from math import ceil
from model_vc import Generator
from pydub import AudioSegment
from make_spect import make_spect_fun

def pad_seq(x, base=32):
    len_out = int(base * ceil(float(x.shape[0])/base))
    len_pad = len_out - x.shape[0]
    assert len_pad >= 0
    return np.pad(x, ((0,len_pad),(0,0)), 'constant'), len_pad

device = 'cuda:0'
G = Generator(28,256,512,16).eval().to(device)

#load model(pre-train and train)
g_checkpoint = torch.load('autovc-zhu.ckpt')
#G = Generator(32,256,512,32).eval().to(device)
#g_checkpoint = torch.load('autovc.ckpt',map_location='cuda:0')
G.load_state_dict(g_checkpoint['model'])
train = pickle.load(open("./spmel/train.pkl",'rb'))
trainchinese = pickle.load(open("./chinesespmel/chinesetrain.pkl",'rb'))

In [4]:
metadata_dataset = []
spect_vc = []
# Test dataset
content_Hotpot1 = np.load('./chinesespmel/S0002/BAC009S0002W0122.npy') 
content_Hotpot2 =np.load('./chinesespmel/S0003/BAC009S0003W0123.npy')
content_Lance1 = np.load('./chinesespmel/S0004/BAC009S0004W0121.npy') 
content_Lance2 = np.load('./chinesespmel/S0005/BAC009S0005W0128.npy')

metadata_dataset.append([train[1][0],train[1][1],content_Hotpot1])
metadata_dataset.append([train[3][0],train[3][1],content_Hotpot2])
metadata_dataset.append([train[7][0],train[7][1],content_Lance2])
metadata_dataset.append([train[4][0],train[4][1],content_Lance1])

for sbmt_i in metadata_dataset:
             
    x_org = sbmt_i[2]
    x_org, len_pad = pad_seq(x_org)

    #sourse content(1 x * x 80)
    uttr_org = torch.from_numpy(x_org[np.newaxis, :, :]).to(device)
    #sourse speaker embedding(1 x 256)
    emb_org = torch.from_numpy(sbmt_i[1][np.newaxis, :]).to(device)
    
    for sbmt_j in metadata_dataset:
        
        #target speaker embedding           
        emb_trg = torch.from_numpy(sbmt_j[1][np.newaxis, :]).to(device)
        
        with torch.no_grad():
            _, x_identic_psnt, _ = G(uttr_org, emb_org, emb_trg)
            
        if len_pad == 0:
            #content with target utterance
            uttr_trg = x_identic_psnt[0, 0, :, :].cpu().numpy()
        else:
            uttr_trg = x_identic_psnt[0, 0, :-len_pad, :].cpu().numpy()
        
        spect_vc.append( ('{}x{}'.format(sbmt_i[0], sbmt_j[0]), uttr_trg) )

with open('chineseresults.pkl', 'wb') as handle:
    pickle.dump(spect_vc, handle)    